In [29]:
tolerancia = 0.0001

nEstagios = 30
# converter p/ mol/L
casDy = [3 - 2.7 * n/(nEstagios) for n in range(nEstagios + 1)]
casHo = [5 - 4.5 * n/(nEstagios) for n in range(nEstagios + 1)]
Hs = [0.8 - 0.1 * n/(nEstagios) for n in range(nEstagios + 1)]

k = casDy[0] + casHo[0] + Hs[0]
rao = 2
aDy = 3.22552
bDy = -0.99999

aHo = 3.05008
bHo = -0.68760

In [2]:
def geraF (listaParametros): 
    ca1n0, ca1n1, ca1n2, ca2n1, ca2n2, a1, b1, rao, k = listaParametros
    if ca1n2 == 0 and ca2n2 == 0:        
        return (
            ca1n1 * ( (k/(3*ca1n1 + 3*ca2n1) )**-a1 + 10**b1 + rao)
            - ca1n0 * rao
        )
    else:
        return (
            ca1n1 * ( (k/(3*ca1n1 + 3*ca2n1) )**-a1 + 10**b1 + rao) 
            - ca1n2 * ( (k/(3*ca1n2 + 3*ca2n2) )**-a1 + 10**b1)
            - ca1n0 * rao
        )

In [3]:
def geraVetorFuncoes():
    
    vetorFuncoes = criaMatrizNula(2*nEstagios, 1)

    for n in range(1, nEstagios + 1):
        if n < nEstagios:
            vetorFuncoes[n - 1][0] = geraF([casDy[n - 1], casDy[n], casDy[n + 1], casHo[n], casHo[n + 1],
                                            aDy, bDy, rao, k])
            vetorFuncoes[nEstagios + n - 1][0] = geraF([casHo[n - 1], casHo[n], casHo[n + 1], casDy[n], casDy[n + 1],
                                                     aHo, bHo, rao, k])
        
        else:
            vetorFuncoes[n - 1][0] = geraF([casDy[n - 1], casDy[n], 0, casHo[n], 0, aDy, bDy, rao, k])
            vetorFuncoes[nEstagios + n - 1][0] = geraF([casHo[n - 1], casHo[n], 0, casDy[n], 0,
                                                     aHo, bHo, rao, k])
            
    return vetorFuncoes

In [4]:
def derivadaPontual(funcao, posicaoIncrementoX, listaArgumentos, incrementoX = 0.0001):
    
    listaArgumentosIncrementada = listaArgumentos[:]
    listaArgumentosIncrementada[posicaoIncrementoX] += incrementoX
    dfdx = ( funcao(listaArgumentosIncrementada) - funcao(listaArgumentos) )/incrementoX
    
    return dfdx 

In [5]:
def criaMatrizNula(nRows, nCols = 'não atribuído'):
    matrizNula = []
    
    if nCols == 'não atribuído':
        nCols = nRows
    
    for i in range(nRows):
        linha_i = []
        
        for j in range(nCols):
            termo_ij = 0
            linha_i.append(termo_ij)
            
        matrizNula.append(linha_i)
        
    return matrizNula

In [6]:
def criaMatrizIdentidade(n):
    matriz = criaMatrizNula(n)
    for i in range(n):
        matriz[i][i] = 1
    
    return matriz

In [7]:
def jacobiana():
    
    assert len(casDy) == len(casHo)
    
    matrizJacobiana = criaMatrizNula(2 * nEstagios) 
    paramsDy = [aDy, bDy, rao, k]
    paramsHo = [aHo, bHo, rao, k]
    
    for n in range(1, nEstagios + 1):
                
        if n == 1:
            listaArgumentosDy = [casDy[n-1], casDy[n], casDy[n+1], casHo[n], casHo[n+1]] + paramsDy
            listaArgumentosHo = [casHo[n-1], casHo[n], casHo[n+1], casDy[n], casDy[n+1]] + paramsHo
            
            matrizJacobiana[n - 1][n - 1] = derivadaPontual(geraF, 1, listaArgumentosDy)
            matrizJacobiana[n - 1][n] = derivadaPontual(geraF, 2, listaArgumentosDy)
            matrizJacobiana[n - 1][nEstagios + n - 1] = derivadaPontual(geraF, 3, listaArgumentosDy)
            matrizJacobiana[n - 1][nEstagios + n] = derivadaPontual(geraF, 4, listaArgumentosDy)

            matrizJacobiana[nEstagios + n - 1][n - 1] = derivadaPontual(geraF, 3, listaArgumentosHo)
            matrizJacobiana[nEstagios + n - 1][n] = derivadaPontual(geraF, 4, listaArgumentosHo)
            matrizJacobiana[nEstagios + n - 1][nEstagios + n - 1] = derivadaPontual(geraF, 1, listaArgumentosHo)
            matrizJacobiana[nEstagios + n - 1][nEstagios + n] = derivadaPontual(geraF, 2, listaArgumentosHo)
            
        elif n == nEstagios:
            listaArgumentosDy = [casDy[n-1], casDy[n], 0, casHo[n], 0] + paramsDy
            listaArgumentosHo = [casHo[n-1], casHo[n], 0, casDy[n], 0] + paramsHo
            
            matrizJacobiana[n - 1][n - 2] = derivadaPontual(geraF, 0, listaArgumentosDy)
            matrizJacobiana[n - 1][n - 1] = derivadaPontual(geraF, 1, listaArgumentosDy)
            matrizJacobiana[n - 1][nEstagios + n - 1] = derivadaPontual(geraF, 3, listaArgumentosDy)

            matrizJacobiana[nEstagios + n - 1][n-1] = derivadaPontual(geraF, 3, listaArgumentosHo)
            matrizJacobiana[nEstagios + n - 1][nEstagios + n-2] = derivadaPontual(geraF, 0, listaArgumentosHo)
            matrizJacobiana[nEstagios + n - 1][nEstagios + n-1] = derivadaPontual(geraF, 1, listaArgumentosHo)
        
        else:
            listaArgumentosDy = [casDy[n-1], casDy[n], casDy[n+1], casHo[n], casHo[n+1]] + paramsDy
            listaArgumentosHo = [casHo[n-1], casHo[n], casHo[n+1], casDy[n], casDy[n+1]] + paramsHo
            
            matrizJacobiana[n - 1][n - 2] = derivadaPontual(geraF, 0, listaArgumentosDy)
            matrizJacobiana[n - 1][n - 1] = derivadaPontual(geraF, 1, listaArgumentosDy)
            matrizJacobiana[n - 1][n] = derivadaPontual(geraF, 2, listaArgumentosDy)
            matrizJacobiana[n - 1][nEstagios + n - 1] = derivadaPontual(geraF, 3, listaArgumentosDy)
            matrizJacobiana[n - 1][nEstagios + n] = derivadaPontual(geraF, 4, listaArgumentosDy)           

            matrizJacobiana[nEstagios + n - 1][n-1] = derivadaPontual(geraF, 3, listaArgumentosHo)
            matrizJacobiana[nEstagios + n - 1][n] = derivadaPontual(geraF, 4, listaArgumentosHo)
            matrizJacobiana[nEstagios + n - 1][nEstagios + n - 2] = derivadaPontual(geraF, 0, listaArgumentosHo)
            matrizJacobiana[nEstagios + n - 1][nEstagios + n - 1] = derivadaPontual(geraF, 1, listaArgumentosHo)
            matrizJacobiana[nEstagios + n - 1][nEstagios + n] = derivadaPontual(geraF, 2, listaArgumentosHo)
        
    return matrizJacobiana

In [8]:
def muda_linha (li, lj, razaoPivotamento): 
    # 'lj' é a linha a ser modificada, 'li' a modificadora e 'i' é o elemento a ser zerado

    li_modificador = []
    
    for elemento in li:
        elemento = elemento * razaoPivotamento
        li_modificador.append(elemento)    

    for i in range(len(lj)):
        lj[i] = lj[i] - li_modificador[i]
        
    return lj

def escalonaInferior (matrizOriginal, matrizInversa): 
    #ela vai escalonar a matriz até zerar a diagonal de baixo

    for n in range(len(matrizOriginal)): #vai passar p/ próximo pivô
        l = 1
        while (l + n) < (len(matrizOriginal)): # vai zerar todos os termos de uma coluna que estão abaixo da diagonal
            razaoPivotamento = matrizOriginal[n + l][n] / matrizOriginal[n][n]
            matrizOriginal[n + l] = muda_linha(matrizOriginal[n], matrizOriginal[n + l], razaoPivotamento)
            matrizInversa[n + l] = muda_linha(matrizInversa[n], matrizInversa[n + l], razaoPivotamento)
            l += 1
        
    return matrizOriginal, matrizInversa
    
def escalonaSuperior (matrizOriginal, matrizInversa): 
    #ela vai escalonar a diagonal de cima, partindo da diagonal de baixo zerada
    
    for n in range(len(matrizOriginal) -1, -1, -1):
        l = 1
        while l <= n: #vai passar p/ pivô anterior
            razaoPivotamento = matrizOriginal[n - l][n] / matrizOriginal[n][n]
            matrizOriginal[n - l] = muda_linha (matrizOriginal[n], matrizOriginal[n - l], razaoPivotamento)
            matrizInversa[n - l] = muda_linha (matrizInversa[n], matrizInversa[n - l], razaoPivotamento)
            l += 1

    return matrizOriginal, matrizInversa

def transformaEscalonadaEmIdentidade (matrizOriginal, matrizInversa):   
   
    for i in range(len(matrizInversa)):
        for j in range(len(matrizInversa)):
            matrizInversa[i][j] = matrizInversa[i][j]/matrizOriginal[i][i]
        
    return matrizOriginal, matrizInversa

def inverteMatriz(matrizOriginal):
    
    matrizInversa = criaMatrizIdentidade(len(matrizOriginal))
    matrizOriginalTriangularSuperior, matrizInversaTriangularSuperior = escalonaInferior(matrizOriginal, matrizInversa)    
    matrizOriginalEscalonada, matrizInversaEscalonada = escalonaSuperior(matrizOriginalTriangularSuperior,
                                                                         matrizInversaTriangularSuperior)
    matrizIdentidade, matrizOriginalInvertida = transformaEscalonadaEmIdentidade(matrizOriginalEscalonada,
                                                                                 matrizInversaEscalonada)
    return matrizOriginalInvertida

In [9]:
def multiplicacaoMatrizes(matrizA, matrizB):
    # A X B = C
    assert len(matrizA[0]) == len(matrizB)
    
    matrizC = criaMatrizNula(len(matrizA), len(matrizB[0]))
    
    for i in range(len(matrizA)):
        for j in range(len(matrizB[0])):
            for n in range(len(matrizB)):
                matrizC[i][j] += matrizA[i][n] * matrizB[n][j]
    
    return matrizC    

In [10]:
def updateVetores(vetorNovo):
    
    for i in range(int(len(vetorNovo)/2)):
        casDy[i + 1] -= vetorNovo[i][0]
        casHo[i + 1] -= vetorNovo[i + nEstagios][0]
    
    return casDy, casHo

In [11]:
def verificaConvergencia(vetor, tolerancia):
    
    for i in range(len(vetor)):
        if vetor[i][0] > tolerancia:
            return False
    
    return True

In [18]:
def newtonRaphsonSistema():
    
    convergencia = False
    n = 0
    
    while not convergencia or n > 100:
        
        matrizJacobiana = jacobiana()
        jacobianaInversa = inverteMatriz(matrizJacobiana)
        vetorFuncoes = geraVetorFuncoes()
        vetorNovo = multiplicacaoMatrizes(jacobianaInversa, vetorFuncoes)
        casDy, casHo = updateVetores(vetorNovo)
        convergencia = verificaConvergencia(vetorNovo, tolerancia)
        if convergencia:
            return casDy, casHo
        n += 1
    
    return 'não convergiu em ' + str(n) + ' iterações.'        

In [13]:
from sympy import *
import numpy as np

def displayMatriz(matriz):
    for i in range(len(matriz)):
        for j in range(len(matriz)):
            matriz[i][j] = round(matriz[i][j],1)
    return Matrix(matriz)

inv = np.linalg.inv(jacobiana())

In [33]:
newtonRaphsonSistema()

([3.0,
  1.5175793746344473,
  1.1919246500508034,
  1.0647581552265277,
  0.9992999016818929,
  0.9597404524296763,
  0.9331852749538286,
  0.9139783129465667,
  0.8992808994989453,
  0.8875211126540785,
  0.8777599312047023,
  0.8694008445806912,
  0.8620444388613167,
  0.8554101078509317,
  0.8492911894550363,
  0.8435277071414713,
  0.8379887421783832,
  0.8325601215424323,
  0.8271348700626823,
  0.8216046941278363,
  0.8158510278157813,
  0.8097339881866307,
  0.803076845969469,
  0.7956419062115353,
  0.7870899417445345,
  0.7769067691816448,
  0.7642594475060975,
  0.747685997576997,
  0.724328147242781,
  0.6875436625692226,
  0.6145757252852944],
 [5.0,
  2.6246123291733734,
  2.07484956930314,
  1.8543731558802001,
  1.739017115534515,
  1.6685211026579787,
  1.6208098097843695,
  1.5860806370288358,
  1.5593678858198485,
  1.537901366233493,
  1.5200160209201057,
  1.504648401284027,
  1.491082844749423,
  1.4788140173079423,
  1.4674677008270176,
  1.4567523700861096,
  1.

In [17]:
displayMatriz(inverteMatriz(jacobiana()))

Matrix([
[ 0.1,  0.1,  0.1,  0.1,  0.0,  0.0,  0.0,  0.0,  0.0, 0.0, 0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0, 0.0],
[ 0.0,  0.1,  0.1,  0.1,  0.1,  0.0,  0.0,  0.0,  0.0, 0.0, 0.0, -0.1, -0.1,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0, 0.0],
[ 0.0,  0.1,  0.2,  0.2,  0.1,  0.1,  0.0,  0.0,  0.0, 0.0, 0.0,  0.0, -0.1, -0.1,  0.0,  0.0,  0.0,  0.0,  0.0, 0.0],
[ 0.0,  0.0,  0.1,  0.3,  0.2,  0.1,  0.1,  0.0,  0.0, 0.0, 0.0,  0.0, -0.1, -0.1, -0.1,  0.0,  0.0,  0.0,  0.0, 0.0],
[ 0.0,  0.0,  0.1,  0.2,  0.4,  0.2,  0.1,  0.0,  0.0, 0.0, 0.0,  0.0,  0.0, -0.1, -0.2, -0.1,  0.0,  0.0,  0.0, 0.0],
[ 0.0,  0.0,  0.1,  0.1,  0.3,  0.5,  0.2,  0.1,  0.0, 0.0, 0.0,  0.0,  0.0, -0.1, -0.1, -0.2,  0.0,  0.0,  0.0, 0.0],
[ 0.0,  0.0,  0.0,  0.1,  0.2,  0.4,  0.5,  0.2,  0.1, 0.0, 0.0,  0.0,  0.0, -0.1, -0.1, -0.2, -0.1,  0.0,  0.0, 0.0],
[ 0.0,  0.0,  0.0,  0.1,  0.2,  0.3,  0.4,  0.6,  0.2, 0.1, 0.0,  0.0,  0.0, -0.1, -0.1, -0.2, -0.1, -0.1,  0.0, 0.0],
[ 0.0,  0.0,  0.0,  0.1,  0.2,  0.3,  0